### **Reading Raw Ingested Data from Bronze layer**

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import *


In [0]:
df_payments = spark.read.format("parquet")\
                .load("abfss://bronze@stretailenvdev.dfs.core.windows.net/f_Payments")

In [0]:
df_payments.limit(5).display()

### **Incremental loading**

In [0]:
silver_path = "abfss://silver@stretailenvdev.dfs.core.windows.net/s_Payments"

from delta.tables import DeltaTable

if DeltaTable.isDeltaTable(spark, silver_path):
    last_max_ts = spark.read.format("delta")\
                    .load(silver_path)\
                    .agg(max("paymentDate").alias("max_ts"))\
                    .first()["max_ts"]
else:
    last_max_ts = None

In [0]:
if last_max_ts:
    df_inc = df_payments.filter(col("PaymentDate") > lit(last_max_ts))
else:
    df_inc = df_payments

**Handling Null**

In [0]:
df_inc = df_inc.fillna({"Amount": 0.0})

### **Upsert (Merge) logic into Silver Delta Table**

In [0]:
if DeltaTable.isDeltaTable(spark, silver_path):
    silver_table = DeltaTable.forPath(spark, silver_path)
    (
        silver_table.alias("tgt")
        .merge(
            df_inc.alias("src"),
            "tgt.paymentID = src.paymentID"
        )
        .whenMatchedUpdateAll()
        .whenNotMatchedInsertAll()
        .execute

    )
else:
    df_inc.write.format("delta").mode("overwrite").save(silver_path)
